In [7]:
import duckdb
import pandas as pd  
import os
from datetime import datetime
from pathlib import Path

In [76]:
con = duckdb.connect(database='dados_duckdb.db', read_only=False)


In [70]:
#pegar o caminho relativo aos dados landing\z0019_1.csv

# garante que o notebook rode mesmo se esta célula for executada antes das imports
arquivo = 'z0019_2.csv'
dados_csv = Path(os.getcwd()).parent/ 'landing' / arquivo

df = pd.read_csv(dados_csv, sep=';', dtype=str)
df['nome_arquivo'] = arquivo
df['data_ingestao'] = datetime.now().strftime('%Y-%m-%d') # adiciona coluna com data e hora da ingestão



print(df.head())



   NATBR     MAKTX WERKS MAINS LABST nome_arquivo data_ingestao
0  10004     SERRA  BT50   100   200  z0019_2.csv    2026-01-09
1  10005  MACHADDO  BT50   100   100  z0019_2.csv    2026-01-09
2  10003     PREGO  BT10   100    60  z0019_2.csv    2026-01-09


In [71]:
con.execute("""
CREATE TABLE IF NOT EXISTS bronze_produtos (
   NATBR VARCHAR,
   MAKTX VARCHAR,
   WERKS VARCHAR,
   MAINST VARCHAR,
   LABST VARCHAR,
   nome_arquivo VARCHAR,
   data_ingestao TIMESTAMP
   )
   """
)


In [72]:
con.execute ("INSERT INTO bronze_produtos SELECT * FROM df")

In [73]:
resultado = con.execute('SELECT * FROM bronze_z0019').fetchdf()

resultado.head()

,NATBR,MAKTX,WERKS,MAINST,LABST,nome_arquivo,data_ingestao
0,10001,PARAFUSO,BT10,100,100,z0019_1.csv,2026-01-09
1,10002,MARTELO,BT50,100,1500,z0019_1.csv,2026-01-09
2,10003,PREGO,BT10,100,50,z0019_1.csv,2026-01-09
3,10004,SERRA,BT50,100,200,z0019_2.csv,2026-01-09
4,10005,MACHADDO,BT50,100,100,z0019_2.csv,2026-01-09


In [79]:
con.commit()
con.close()